In [1]:
# Do not delete this cell. It ensures that you can do the imports,
# load datasets etc. in the same fashion as in any Python script
# in the project template.


import sys
sys.path.insert(0, '../..')
from bld.project_paths import project_paths_join as ppj
from bld.project_paths import project_paths as pp

%config Completer.use_jedi = False

#see https://github.com/yaroslavrosokha/sfem/blob/master/sfem.ipynb

In [2]:
import numpy as np
import json
import pickle
import pandas as pd
from scipy.stats import sem
import seaborn as sns
import matplotlib.pyplot as plt
%config Completer.use_jedi = False

from matplotlib import rc
rc('text', usetex=True)
plt.rcParams.update({'font.size': 20})


In [10]:
"""Always cooperate strategy"""
actions  = [0,1,1]
n = len(actions)
out = np.ones(n)
out[np.isnan(actions)] = np.nan

In [13]:
np.isnan([1,2])

array([False, False])

In [14]:
data = pd.read_pickle(ppj("OUT_DATA", "data_individual_level.pickle"))

In [43]:
data_last_sg = data.loc[(data['super_game'] == 3) &
                             (data['treatment'] == '1H1A') ].copy()
# data_last_sg_1H2A = data.loc[(data['super_game'] == 1) &
#                              (data['treatment'] == '1H2A') ]

In [44]:
list(data_last_sg)

['participant.code',
 'round',
 'super_game',
 'group_id',
 'winning_price',
 'price',
 'price_algorithm',
 'data_identifier',
 'treatment',
 'super_group_id',
 'unique_prices_humans',
 'collusive',
 'super_group_id_general',
 'group_id_general',
 'winning_price_lag_0',
 'price_lag_0',
 'price_algorithm_0',
 'winning_price_lag_1',
 'price_lag_1',
 'price_algorithm_1',
 'winning_price_lag_2',
 'price_lag_2',
 'price_algorithm_2',
 'winning_price_lag_3',
 'price_lag_3',
 'price_algorithm_3',
 'winning_price_lag_4',
 'price_lag_4',
 'price_algorithm_4',
 'r_sg']

Reshape the data to wide format

In [45]:
def always_cooperate(p, p_other_lag, rounder_number):
    return 1 if p == 4 else 0

def always_defect(p, p_other_lag, rounder_number):
    return 1 if p == 1 else 0

def exploit(p, p_other_lag, rounder_number):
    if p == 3 and p_other_lag == 1:
        return 1
    elif p == 1 and p_other_lag == 4:
        return 1
    elif rounder_number == 1 and p == 3:
        return 1
    else:
        return 0

In [46]:
def exploit_at_2(p, p_other_lag, rounder_number):
    if p == 2 and p_other_lag == 1:
        return 1
    elif p == 1 and p_other_lag == 4:
        return 1
    elif rounder_number == 1 and p == 2:
        return 1
    else:
        return 0

In [47]:
data_dal = data_last_sg[['participant.code', 'round', 'price', 'price_lag_1', 'price_algorithm_1']].copy()

In [48]:
data_dal['ac'] = data_dal.apply(lambda x: always_cooperate(x['price'], x['price_algorithm_1'], x['round']), axis=1)

In [49]:
data_dal['ad'] = data_dal.apply(lambda x: always_defect(x['price'], x['price_algorithm_1'], x['round']), axis=1)

In [50]:
data_dal['exploit'] = data_dal.apply(lambda x: exploit(x['price'], x['price_algorithm_1'], x['round']), axis=1)

In [51]:
data_dal['exploit_at_2'] = data_dal.apply(lambda x: exploit_at_2(x['price'], x['price_algorithm_1'], x['round']), axis=1)

In [52]:
data_agg_sum = data_dal.groupby(['participant.code'], as_index=False).sum()

In [53]:
correct_matrix = data_agg_sum[['exploit', 'ac', 'ad', 'exploit_at_2']].values

In [54]:
incorrect_matrix = np.ones(correct_matrix.shape)* data_dal['round'].max() - correct_matrix

In [55]:
def objective(x, args):
    C = args[0]
    E = args[1]
    
    bc=np.power(x[0],C) #beta to the power of C
    be=np.power(1-x[0],E) #beta to the power of E
    prodBce = np.multiply(bc,be) #Hadamard product
    
    #maximum is taken so that there is no log(0) warning/error
    res = np.log(np.maximum(np.dot(prodBce, x[1:]),np.nextafter(0,1))).sum() 
    
    return -res

In [56]:
from scipy.optimize import minimize

In [57]:
def constraint1(x):
    
    return x[1:].sum()-1


n_strats = 4

#Set up the boundaries and constraints
b0 = (np.nextafter(0.5,1),1-np.nextafter(0,1))
b1 = (np.nextafter(0,1),1-np.nextafter(0,1))
bnds = tuple([b0]+[b1]*n_strats) #Beta is at least .5
con1 = {'type': 'eq', 'fun': constraint1} 
cons = ([con1])



In [79]:
np.random.seed(9)
n_bootstrap = 10
estimates_bootstrap = np.zeros((n_bootstrap, n_strats+1))
n_participants = correct_matrix.shape[0] 

for b in range(n_bootstrap):
    #Some random starting point
    x0 = np.zeros(n_strats+1)
    x0[0] = .5+.5*np.random.random()
    temp = np.random.random(n_strats)
    x0[1:]=temp/temp.sum()
    
    # create bootstrap sample
    index_sample = np.random.randint(0, n_participants, n_participants)
    correct_boot = correct_matrix[index_sample, :]
    incorrect_boot = incorrect_matrix[index_sample, :]

    bestX=x0
    bestObjective=objective(x0,[correct_boot, incorrect_boot])

    for k in range(50): #Do many times so that there is low chance of getting stuck in local optimum
        x0 = np.zeros(n_strats+1)
        x0[0] = .5+.5*np.random.random()
        temp = np.random.random(n_strats)
        x0[1:]=temp/temp.sum()

        #Notice that we are minimizing the negative
        solution = minimize(objective,x0,method='SLSQP',bounds=bnds,constraints=cons,args=([correct_boot, incorrect_boot]))
        x = solution.x
        obj = solution.fun

        if bestObjective>obj:
            bestObjective=obj
            bestX=x
    estimates_bootstrap[b,:] = x

In [77]:
np.random.randint(0, 1, 2, )

array([0, 0])

In [59]:
#Some random starting point
x0 = np.zeros(n_strats+1)
x0[0] = .5+.5*np.random.random()
temp = np.random.random(n_strats)
x0[1:]=temp/temp.sum()

bestX=x0
bestObjective=objective(x0,[correct_matrix, incorrect_matrix])

for k in range(50): #Do many times so that there is low chance of getting stuck in local optimum
    x0 = np.zeros(n_strats+1)
    x0[0] = .5+.5*np.random.random()
    temp = np.random.random(n_strats)
    x0[1:]=temp/temp.sum()

    #Notice that we are minimizing the negative
    solution = minimize(objective,x0,method='SLSQP',bounds=bnds,constraints=cons,args=([correct_matrix, incorrect_matrix]))
    x = solution.x
    obj = solution.fun

    if bestObjective>obj:
        bestObjective=obj
        bestX=x
print("Estimates: ", x)

Estimates:  [9.34639475e-01 2.90342040e-01 6.12952634e-01 9.67053256e-02
 1.73728945e-12]


In [21]:
np.std(estimates_bootstrap, axis=0)

array([0.09942139, 0.13874018, 0.1628809 , 0.08895986, 0.09288215])

In [22]:
['beta', 'exploit', 'ac', 'ad', 'exploit_at_2']

['beta', 'exploit', 'ac', 'ad', 'exploit_at_2']

In [23]:
# g = sns.FacetGrid(data_dal, col="participant.code", height=3.5, aspect=1, col_wrap=3)
# g.map_dataframe(sns.lineplot, x="round", y="price")
